In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install --upgrade bitsandbytes
!pip install rouge_score
!pip install nltk
!pip install bert_score
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=94de2dc40bb4a835a9f1973688ef29c05fd03e83ab0cdbb380d3fcf4c4735d11
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import transformers
from peft import PeftModel

import torch
import pandas as pd
import random

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths
base_model_path = "mistralai/Mistral-7B-v0.1"  # Path to your base model
#model_id = "meta-llama/Meta-Llama-3-8B"
#model_id = "mistralai/Mistral-7B-v0.1"
lora_weights_path = "/content/drive/MyDrive/fine_tuned_model/mistral-7b"  # Path to your fine-tuned LoRA weights

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    trust_remote_code=True,   # Allow downloading of custom code files if needed
    device_map="auto"         # Automatically map model layers to available devices, can be omitted if using .to("cuda")
).to("cuda")

# Load the tokenizer from your fine-tuned model path instead of the base model path
tokenizer = AutoTokenizer.from_pretrained(lora_weights_path, local_files_only=True)
#tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Resize embeddings (Ensure the base model has the same vocab size as your fine-tuned model)
base_model.resize_token_embeddings(len(tokenizer))

# Apply the LoRA weights to the base model
model = PeftModel.from_pretrained(
    base_model,
    lora_weights_path,
    local_files_only=True
).to("cuda")

print("Fine-tuned model loaded successfully!")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Fine-tuned model loaded successfully!


In [ ]:
# Print out the layers that require gradients
for name, param in model.named_parameters():
    if "lora" in name.lower():  # Assuming the LoRA layers contain 'lora' in their names
        print(f"{name}: requires_grad={param.requires_grad}")


base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: requires_grad=False
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: requires_grad=False
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight: requires_grad=False
base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight: requires_grad=False
base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight: requires_grad=False

# Evaluate 5 question manually

In [ ]:
# Set the model to evaluation mode
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.3, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [ ]:
# Load the evaluation dataset
df = pd.read_csv('/content/medquad.csv')
# Remove rows with NaN values in the 'answer' column
df = df.dropna(subset=['answer'])
# Strip whitespace and convert to lowercase for both the dataset and selected questions
df['question'] = df['question'].str.strip().str.lower()
selected_questions = [
    "what is (are) paget's disease of bone ?",
    "what are the treatments for balance problems ?",
    "what are the treatments for hearing loss ?",
    "how to diagnose osteoporosis ?",
    "what causes dry mouth ?",
]

# Filter using standardized question list
evaluation_questions = df[df['question'].isin(selected_questions)][['question', 'answer']]
# Display the resulting filtered DataFrame
evaluation_questions


,question,answer
33,what is (are) paget's disease of bone ?,Enlarged and Misshapen Bones Paget's disease o...
37,what is (are) paget's disease of bone ?,Paget's disease of bone is a disease that caus...
151,what are the treatments for balance problems ?,Your doctor can recommend strategies to help r...
160,what are the treatments for balance problems ?,In BPPV (benign paroxysmal positional vertigo)...
161,what are the treatments for balance problems ?,Mnire's disease is caused by changes in fluid ...
196,what causes dry mouth ?,People get dry mouth when the glands in the mo...
200,what causes dry mouth ?,"Dry mouth can cause several problems, includin..."
202,what causes dry mouth ?,"Yes. More than 400 medicines, including some o..."
204,what causes dry mouth ?,Some diseases affect the salivary glands. Sjgr...
213,how to diagnose osteoporosis ?,Who Should Be Tested? The United States Preven...


In [ ]:
# prompt: select 5 unique questions from evaluation_questions

# Select 5 unique random questions from the evaluation_questions DataFrame
selected_rows = evaluation_questions.sample(n=5)

# Print the selected questions
print(selected_rows)


                                        question  \
33       what is (are) paget's disease of bone ?   
1848     what is (are) paget's disease of bone ?   
202                      what causes dry mouth ?   
288   what are the treatments for hearing loss ?   
213               how to diagnose osteoporosis ?   

                                                 answer  
33    Enlarged and Misshapen Bones Paget's disease o...  
1848  Paget's disease of bone causes your bones to g...  
202   Yes. More than 400 medicines, including some o...  
288   Your doctor can recommend strategies to help r...  
213   Who Should Be Tested? The United States Preven...  


In [ ]:
def balanced_generation(question, max_length=256):
    prompt = question.strip()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Generate output using the model with beam search
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        num_beams=5,                # Use beam search for better quality control
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.8,      # Increase repetition penalty
    )

    # Decode the output
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer.strip()


In [ ]:
# Generate answers for evaluation questions
selected_rows['generated_answer'] = selected_rows['question'].apply(balanced_generation)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# prompt: print out all the rows in evaluations_questions in the following format: "question" first, "generated_answer" in new line, "answer" in last line, with each set print out seperate by line
for index, row in selected_rows.iterrows():
  print(f"User: {row['question']}\n")
  print(f"Generated Answer: {row['generated_answer']}\n")
  print(f"Actual Answer: {row['answer']}\n")
  print("-"*50)

User: what is (are) paget's disease of bone ?

Generated Answer: what is (are) paget's disease of bone ?

Paget’s disease, also known as osteitis deformans, is a chronic skeletal disorder characterized by abnormal bone resorption and formation. It is named after Sir James Paget, an English surgeon who first described the disease in 1877.

The disease primarily affects the long bones of the body, such as the femur, tibia, humerus, and vertebrae. It can also affect the skull, pelvis, ribs, and other bones. In some cases, it can lead to bone deformities, fractures, and arthritis. The exact cause of the disease is not fully understood, but it is believed to be related to a combination of genetic and environmental factors.

Actual Answer: Enlarged and Misshapen Bones Paget's disease of bone causes affected bones to become enlarged and misshapen. Our bones are living tissue, and our bodies are constantly breaking down old bone and replacing it with new bone. In Paget's disease, however, old 

# Evaluate 200 Question after Fine-Tuning

In [ ]:
# Choose a fixed random seed for reproducibility
random.seed(42)
# Display the evaluation_subset DataFrame to check generated answers
num_questions = 200  # Adjust this number as needed
evaluation_subset = df.sample(n=num_questions, random_state=42).reset_index(drop=True)

In [ ]:
# Apply the generate_answer function to the selected subset
evaluation_subset['generated_answer'] = evaluation_subset['question'].apply(balanced_generation)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
# Save the relevant columns to a CSV file
evaluation_subset[['question','answer', 'generated_answer']].to_csv('generated_answers.csv', index=False)

print("CSV file saved successfully as 'generated_answers.csv'.")

CSV file saved successfully as 'generated_answers.csv'.


In [ ]:
from google.colab import files
files.download('generated_answers.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Initialize the ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Function to calculate scores
def calculate_scores(row):
    reference = row['answer']
    generated = row['generated_answer']

    # Calculate BLEU score
    bleu = sentence_bleu([reference.split()], generated.split())

    # Calculate ROUGE scores
    rouge_scores = rouge.score(reference, generated)

    # Calculate BERTScore (useful for semantic similarity)
    P, R, F1 = bert_score([generated], [reference], lang="en", rescale_with_baseline=True)

    return pd.Series({
        'bleu_score': bleu,
        'rouge1_f1': rouge_scores['rouge1'].fmeasure,
        'rougeL_f1': rouge_scores['rougeL'].fmeasure,
        'bert_score_f1': F1.mean().item()
    })

# Apply scoring function to the evaluation subset
evaluation_subset[['bleu_score', 'rouge1_f1', 'rougeL_f1', 'bert_score_f1']] = evaluation_subset.apply(calculate_scores, axis=1)

# Display the evaluation scores
print(evaluation_subset[['bleu_score', 'rouge1_f1', 'rougeL_f1', 'bert_score_f1']].describe())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-package

          bleu_score   rouge1_f1   rougeL_f1  bert_score_f1
count   2.000000e+02  200.000000  200.000000     200.000000
mean    1.627458e-02    0.296833    0.165698      -0.018007
std     4.420917e-02    0.098582    0.064237       0.157555
min    7.411995e-232    0.061728    0.037037      -0.384908
25%    3.031011e-155    0.230177    0.131422      -0.130240
50%     9.105286e-79    0.298660    0.159437      -0.011908
75%     2.176140e-02    0.358031    0.188873       0.087581
max     3.900382e-01    0.652482    0.638298       0.533081


In [ ]:
average_bleu = evaluation_subset['bleu_score'].mean()
average_rouge1_f1 = evaluation_subset['rouge1_f1'].mean()
average_rougeL_f1 = evaluation_subset['rougeL_f1'].mean()
average_bert_score_f1 = evaluation_subset['bert_score_f1'].mean()

print(f"Average BLEU Score: {average_bleu}")
print(f"Average ROUGE-1 F1 Score: {average_rouge1_f1}")
print(f"Average ROUGE-L F1 Score: {average_rougeL_f1}")
print(f"Average BERTScore F1: {average_bert_score_f1}")

Average BLEU Score: 0.016274583883031827
Average ROUGE-1 F1 Score: 0.2968329677125279
Average ROUGE-L F1 Score: 0.16569820806648153
Average BERTScore F1: -0.018006965910317377




```
`# This is formatted as code`
```

